In [2]:
import pandas as pd
data1 = pd.read_csv("Mayuka_output_120217.csv",
                         keep_default_na=False, na_values=[""])

data2 = pd.read_csv("Mayuka_output2_120217.csv",
                         keep_default_na=False, na_values=[""])

data3 = pd.read_csv("Mayuka_output3_120217.csv",
                         keep_default_na=False, na_values=[""])

data4 = pd.read_csv("Mayuka_output4_120217.csv",
                         keep_default_na=False, na_values=[""])

data5 = pd.read_csv("Mayuka_output5_120217.csv",
                         keep_default_na=False, na_values=[""])
data6 = pd.read_csv("Mayuka_output1_120317.csv",
                         keep_default_na=False, na_values=[""])
data7 = pd.read_csv("Mayuka_output2_120317.csv",
                         keep_default_na=False, na_values=[""])
data_agg = pd.concat([data2, data3])
data_agg = pd.concat([data_agg, data4])
data_agg = pd.concat([data_agg, data5])
data_agg = pd.concat([data_agg, data6])
data_agg = pd.concat([data_agg, data7])
data_agg.to_csv(path_or_buf="aggregatefile_pandas.csv")

In [32]:
from ast import literal_eval
dataFromFile = {}
addresses = []
complaint_history = pd.read_csv("aggregatecounts_311complaints.csv", keep_default_na=False, na_values=[""])
with open("aggregatefile_pandas.csv", "r") as f:
    next(f)
    for line in f:
        key = literal_eval(line[line.find("("):line.find(")")+1])
        addresses.append(key)
        value = literal_eval(line[line.find(")")+1:])
        
        #add feature for number of 311 complaints: (buggy??)
        complaint_feature = 0
        if complaint_history["incident_address"].str.contains(key[0]).all():
            complaint_feature = complaint_history['count'].where(complaint_history['incident_address'] == key[0])
        value.append(complaint_feature)
        
        dataFromFile[key] = value

In [33]:
limit = 0
for i in dataFromFile:
    print (dataFromFile[i])
    if limit > 10: break
    limit += 1

[6.0, 40.645540000000004, -73.99869, 2016, 1931, 1916, 4336, 20000403, 460000, 1491041, 20171130, 32915, 1789249, 1789249, 11232.0]
[1.0, 40.61745, -73.98947, 2016, 1930, 2667, 2600, 20121211, 800000, 1422270, 20171130, -38433, 1521829, 1521829, 11204.0]
[1.0, 40.677665000000005, -73.862635, 2016, 1925, 2383, 2312, 20041214, 510000, 656934, 20171130, 3002, 689781, 689781, 11208.0]
[6.0, 40.637229999999995, -73.90092, 2016, 1950, 2244, 2838, 20130607, 427450, 660945, 20171130, 2221, 693992, 693992, 11236.0]
[2.0, 40.883905, -73.86456, 2016, 2006, 1215, 2670, 20070927, 685000, 604218, 20171130, -4958, 652555, 652555, 10467.0]
[6.0, 40.54347, -74.166013, 2016, 2002, 2891, 2050, 20130926, 425000, 611989, 20171130, -2414, 648708, 648708, 10312.0]
[6.0, 40.676495, -73.90858, 2016, 1910, 1350, 2675, 20070119, 670000, 985714, 20171130, -21235, 1064571, 1064571, 11233.0]
[1.0, 40.64127, -73.91296, 2016, 1960, 2262, 1800, 20061017, 545000, 541537, 20171130, 2610, 568614, 568614, 11236.0]
[2.0, 4

In [34]:
#Partition X and Y
xData = []
yData = []
labels = ['home_type', 'latitude', 'longitude', 'tax_year','year_built','property_size', 'home_size', 'last_sold_date', 'last_sold_price', 'zestimate_amount', 'zestimate_last_updated', 'zestimate_value_change', 'zestimate_valuation_range_low', 'zestimate_valuation_range_high']
for key in dataFromFile:
    featuresAndLabel = list(key) + dataFromFile[key]
    xData.append(featuresAndLabel[:-7] + featuresAndLabel[-6:])
    yData.append(int(featuresAndLabel[-7]))

In [35]:
#Remove Address and State
newXData = []
for x in xData:
    newDp = x[2:]
    newXData.append(newDp)
xData = newXData

In [36]:
import numpy as np
from sklearn.model_selection import KFold
X = np.array(xData)
y = np.array(yData)
kf = KFold(n_splits=4)
kf.get_n_splits(X)
KFold(n_splits=2, random_state=None, shuffle=False)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [37]:
def RMSD(predVals, trueVals):
    sum = 0
    for i in range(len(trueVals)):
        sum += (trueVals[i] - predVals[i])**2
    return sum**0.5

In [38]:
#Calculate R-squared

def avg(trueVals):
    sum = 0
    for i in range(len(trueVals)):
        sum+= trueVals[i]
    return sum/(len(trueVals)*1.0)

def rsquared(predVals, trueVals):
    SSE = 0
    TSS = 0
    avgVal = avg(trueVals)
    for i in range(len(trueVals)):
        SSE += (trueVals[i] - predVals[i])**2
        TSS += (trueVals[i] - avgVal)**2
    return 1 - (SSE/TSS)
    

In [39]:
def MPE(predVals, trueVals):
    err = 0
    for i in range(len(trueVals)):
        err += abs((trueVals[i] - predVals[i])/trueVals[i])
    return err/(len(trueVals)*1.0)

In [40]:
from sklearn import linear_model
linear_model = linear_model.LinearRegression()
linear_model.fit(X_train, y_train)
print (RMSD(linear_model.predict(xData), yData)/(len(yData)))
print(linear_model.coef_)
print (rsquared(linear_model.predict(X_test), y_test))
print(len(X_test))
#with open("ouputs.txt", "w") as f:
    #outputs = linear_model.predict(xData)
    #for i in range(len(addresses)-1):
        #f.write(str(addresses[i]) + "\t" + str(outputs[i])+"\n")

16166.1344016
[ -3.04943822e+04  -1.81071142e+06   1.77165644e+06  -7.48560973e+04
   2.59027886e+03  -7.04903505e-05  -7.40980211e+00   2.72105110e+00
  -8.14044946e-01  -2.21317878e+02  -3.85553256e+00   6.46675046e-01
   6.46900361e-01  -3.75125907e+02]
0.3122881426
296


In [42]:
from sklearn import linear_model
lasso_model = linear_model.Lasso(alpha = 0.1)
lasso_model.fit(X_train, y_train)
print (RMSD(lasso_model.predict(xData), yData)/(len(yData)))
print (rsquared(lasso_model.predict(xData), yData))
print(MPE(lasso_model.predict(x_Test), y_Test))
print(lasso_model.coef_)

16166.1338525
0.76974326972


/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


NameError: name 'x_Test' is not defined

In [44]:
from sklearn import linear_model
elastic_model = linear_model.ElasticNet()
elastic_model.fit(X_train, y_train)
print(elastic_model.coef_)
print (RMSD(elastic_model.predict(xData), yData)/(len(yData)))
print (rsquared(elastic_model.predict(X_test), y_test))

[ -2.56588327e+04  -3.76186026e+03   2.44846683e+03  -6.09977676e+02
   3.08753566e+03   6.45764231e-04  -1.05982403e+01   2.69943608e+00
  -9.32421232e-01  -3.20472407e+02  -3.76789494e+00   1.37961640e+00
   3.02326280e-10  -7.08426066e+01]
16328.0133752
0.275433812973


/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [ ]:
from sklearn import linear_model
ridge_model = linear_model.BayesianRidge()
ridge_model.fit(X_train, y_train)
print (RMSD(ridge_model.predict(xData), yData)/(len(yData)))
print (rsquared(ridge_model.predict(X_test), y_test))
print(ridge_model.coef_)

In [ ]:
#print lasso_model.coef_

In [ ]:
from sklearn import linear_model
lasso_model = linear_model.Lasso(alpha = 0.1)
lasso_model.fit(X_train, y_train)
def getPredictions(testPoints, regressionType, xData, yData):
    print (lasso_model.predict([xData[0],xData[2],xData[3],xData[4],xData[5]]))

In [ ]:
testPoints = [
    ('1635 MONTGOMERY AVE', 'New York'),
    ('1721 MAHAN AVE', 'New York'),
    ('931 BURKE AVE', 'New York'),
    ('1256 STRATFORD AVE', 'New York'),
    ('1774 EDENWALD AVE', 'New York')
]
getPredictions(testPoints, 'lasso', X_train, y_train)